## Load packages and helper functions

In [1]:
using LinearAlgebra
using Pipe

"""
Returns rows whose sums are not unity. 
"""
function sanity_check_prob(P::Matrix{T}; tol::Float64 = 1E-5) where T
    return @pipe filter(row_ind -> abs(sum(P[row_ind,:]) - 1.0) > tol, 1:size(P,1)) |> map(bad_row -> (bad_row, sum(P[bad_row,:])), _)
end

function sanity_check_prob(x::Vector{T}; tol::Float64 = 1E-5) where T
    if abs(sum(x) - 1.0) > tol
        println("Invalid probabilities")
    else
        println("Valid probabilities")
    end
end

sanity_check_prob (generic function with 2 methods)

## Construct parameters

Discount factor

In [2]:

#### Discount factor
# %f
discount = 0.99

0.99

Min/Max

In [3]:
#### Min/Max
# [ reward, cost ]
# values =  "cost" # to minimize
values = "reward" # to maximize

"reward"

### States

In [4]:
#### States
# [ %d, <list-of-states> ]
# When an integer is given, the states are enumerated starting from 0
# Delimeters are white space
# Since pomdp-solver requires stationary parameters, we will consider time as a component of the state
# Hence each state consists of ΔNTCP, b, and t
ΔNTCP_states = 0:12
budget = 3
horizon = 4  # 4 decision epochs; this is not the pomdp-solve optional parameter

states = String[]
for t = 1:(horizon+1), b in budget:-1:0, ΔNTCP in ΔNTCP_states
    push!(states, "$(ΔNTCP)_$(b)_$t")
end
push!(states, "Forbidden")  # add dummy absorbing state for when replanning is not allowed

num_ΔNTCP_states = length(ΔNTCP_states)
num_budget_states = budget + 1
num_states = length(states)

261

In [ ]:
# Display states
states

### Actions

In [6]:
#### Actions
# [ %d, <list-of-actions> ]
# When a list is given, the elements may be referred to by name or by 0-based indexing
actions = ["Replan", "Continue"]

2-element Vector{String}:
 "Replan"
 "Continue"

### Observations

In [7]:
#### Observations
# [ %d, <list-of-observations> ]
# Delimeters are white space
# An observation is observed after the state transitions
# Will assume observations are ordered from best to worst, with high pain being worse than a high BMI drop
observations = String[]
levels = ["Low", "Med", "High"]
for pain_level in levels, bmi_level in levels
    push!(observations, "Pain-$(pain_level)___BMI-$(bmi_level)")
end
push!(observations, "Dummy")
# observations = [ "Pain-High___BMI-High", "Pain-High___BMI-Med", "Pain-High___BMI-Low", 
#     "Pain-Med___BMI-High", "Pain-Med___BMI-Med", "Pain-Med___BMI-Low",
#     "Pain-Low___BMI-High", "Pain-Low___BMI-Med", "Pain-Low___BMI-Low"]

num_observations = length(observations)

10

In [8]:
# Display observations
observations

10-element Vector{String}:
 "Pain-Low___BMI-Low"
 "Pain-Low___BMI-Med"
 "Pain-Low___BMI-High"
 "Pain-Med___BMI-Low"
 "Pain-Med___BMI-Med"
 "Pain-Med___BMI-High"
 "Pain-High___BMI-Low"
 "Pain-High___BMI-Med"
 "Pain-High___BMI-High"
 "Dummy"

### State transition probabilities

Optional starting state

In [29]:
#### Optional starting state 
# Since first decision epoch is at F10, the starting probabilities (at full budget and start of horizon) should be the following:
ΔNTCP_start_dist = [0.5, 0.13, 0.08, 0.11, 0.04, 0.04, 0.0, 0.02, 0.0, 0.0, 0.02, 0.02, 0.04]
# This corresponds to the states at the beginning
start_dist =[ΔNTCP_start_dist; zeros(num_states - num_ΔNTCP_states)];

Check probabilities are valid

In [10]:
sanity_check_prob(ΔNTCP_start_dist)

Valid probabilities


ΔNTCP transitions under Replanning

In [30]:
#### Toxicity state transition probabilities
## Replan, F0 to F10
# THIS IS NOT A DECISION EPOCH
# Replan isn't an option at time 0. The trans. prob. of Continue at F0 correspond to the starting distribution
# of the POMDP (see starting state dist. above)

## Replan, F10 to F15, table B.3
T_ΔNTCP_F10toF15_R = [
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0
]

## Replan, F15 to F20, table B.4
T_ΔNTCP_F15toF20_R = [
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
]

## Replan, F20 to F25, table B.5
T_ΔNTCP_F20toF25_R = [
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
]

## Replan, F25 to F30, table B.6
T_ΔNTCP_F25toF30_R = [
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
]

# Store them in a vector
T_ΔNTCP_all_R = [T_ΔNTCP_F10toF15_R, T_ΔNTCP_F15toF20_R, T_ΔNTCP_F20toF25_R, T_ΔNTCP_F25toF30_R]

4-element Vector{Matrix{Float64}}:
 [0.88 0.0 … 0.0 0.0; 0.88 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

ΔNTCP transitions under No Replan (i.e., Continuing)

In [31]:
## Continue, F0 to F10, table B.1
# THIS IS NOT A DECISION EPOCH
# The trans. prob. of Continue at F0 correspond to the starting distribution of the POMDP (see starting state dist. above)
## Continue, F10 to F15, table B.2
T_ΔNTCP_F10toF15_C = [
    0.88 0.0 0.12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.25 0.75 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.5 0.33 0.17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0;
    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.5
]
## Continue, F15 to F20, table B.2
T_ΔNTCP_F15toF20_C = copy(T_ΔNTCP_F10toF15_C)
## Continue, F20 to F25, table B.2
T_ΔNTCP_F20toF25_C = copy(T_ΔNTCP_F10toF15_C)
## Continue, F25 to F30, table B.2
T_ΔNTCP_F25toF30_C = copy(T_ΔNTCP_F10toF15_C)

# Store them in a vector
T_ΔNTCP_all_C = [T_ΔNTCP_F10toF15_C, T_ΔNTCP_F15toF20_C, T_ΔNTCP_F20toF25_C, T_ΔNTCP_F25toF30_C]

4-element Vector{Matrix{Float64}}:
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.5 0.5]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.5 0.5]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.5 0.5]
 [0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.5 0.5]

In [32]:
# Collect into a dictionary for ease of access by Action
T_ΔNTCP_all = Dict("Replan" => T_ΔNTCP_all_R, "Continue" => T_ΔNTCP_all_C)

Dict{String, Vector{Matrix{Float64}}} with 2 entries:
  "Continue" => [[0.88 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "Replan"   => [[0.88 0.0 … 0.0 0.0; 0.88 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0…

#### Create master state transition probability matrix

In [33]:
#### Full state transition probabilities
# T: <action> : <start-state> : <end-state> %f

T = zeros(length(actions), num_states, num_states);

Place probability submatrices into the master state transition matrix

In [34]:
for a_ind in eachindex(actions)
    # println("\nAction: $(actions[a_ind])")
    for t = 1:horizon
        # println("Time = $t")
        # b serves as a counter here
        if actions[a_ind] == "Replan"
            for b = 1:num_budget_states-1
                row_indices = ((t-1)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t-1)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
                col_indices = ((t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states +1):((t)*num_budget_states*num_ΔNTCP_states +(b+1)*num_ΔNTCP_states)
                # println("\trow indices: $row_indices")
                # println("\tcol indices: $col_indices")
                T[a_ind, row_indices, col_indices] = T_ΔNTCP_all[actions[a_ind]][t]
            end
        elseif actions[a_ind] == "Continue"
            for b = 1:num_budget_states
                row_indices = ((t-1)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t-1)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
                col_indices = ((t)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
                # println("\trow indices: $row_indices")
                # println("\tcol indices: $col_indices")
                T[a_ind, row_indices, col_indices] = T_ΔNTCP_all[actions[a_ind]][t]
            end
        end
    end
end

Add remaining probabilities

In [35]:
## Absorbing probabilities
# Whenever the budget is 0 and action is Replan, transition to the absorbing state
for t = 1:horizon
    ind_start = (t-1)*num_ΔNTCP_states*num_budget_states + (num_budget_states-1)*num_ΔNTCP_states + 1
    ind_end = (t-1)*num_ΔNTCP_states*num_budget_states + (num_budget_states)*num_ΔNTCP_states
    # println("Range: $ind_start:$ind_end")
    # println("States: $(states[ind_start:ind_end])")
    T[1, ind_start:ind_end, end] .= 1.0  # last state is the absorbing state
end

## Recursion of the absorbing state
T[1,end,end] = 1.0
T[2,end,end] = 1.0

## Identity probabilities at horizon+1 for both actions, since this is not a real decision epoch
range = horizon *num_ΔNTCP_states *num_budget_states +1:(horizon+1) *num_ΔNTCP_states *num_budget_states
T[1,range, range] = Matrix{Int64}(I(num_ΔNTCP_states*num_budget_states))
T[2,range, range] = Matrix{Int64}(I(num_ΔNTCP_states*num_budget_states));

52×52 Matrix{Int64}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱  ⋮              ⋮              ⋮  
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0

Check probabilities are valid

In [36]:
bad_rows_R = sanity_check_prob(T[1,:,:])
bad_rows_C = sanity_check_prob(T[2,:,:])
println("bad_rows_R: $bad_rows_R\nbad_rows_C: $bad_rows_C")

bad_rows_R: Tuple{Int64, Float64}[]
bad_rows_C: Tuple{Int64, Float64}[]


Write state transition matrix to an excel file (with labels)

In [ ]:
# Write state transition matrix to an excel file (with labels)
# TODO

### Observation Probabilities

#### Create time- and budget-independent (ΔNTCP, obs) submatrix 

In [11]:
# Will assume the observation probabilities are independent of budget and time (may consider time dependence)
# The worse a ΔNTCP state, the likelier for a worse observation
# Will assume observations are ordered from best to worse, with high pain being worse than a high BMI drop

# Observation probabilities should be independent of the actions, what matters is the new state
# The action influences the state transition probabilities

# The following values are made up for testing
O_sub = [
    1/6 1/6 1/6 1/6 1/12 1/12 1/12 1/24 1/24;
    1/6 1/6 1/6 1/6 1/12 1/12 1/12 1/24 1/24;
    1/6 1/6 1/6 1/6 1/12 1/12 1/12 1/24 1/24;
    5/65 5/65 8/52 8/52 8/52 8/52 5/65 5/65 5/65;
    5/65 5/65 8/52 8/52 8/52 8/52 5/65 5/65 5/65;
    5/65 5/65 5/65 8/52 8/52 8/52 8/52 5/65 5/65;
    5/65 5/65 5/65 8/52 8/52 8/52 8/52 5/65 5/65;
    5/65 5/65 5/65 8/52 8/52 8/52 8/52 5/65 5/65;
    1/25 2/25 2/25 2/25 4/25 4/25 4/25 4/25 2/25;
    1/25 2/25 2/25 2/25 4/25 4/25 4/25 4/25 2/25;
    1/24 1/24 1/12 1/12 1/12 1/6 1/6 1/6 1/6;
    1/24 1/24 1/12 1/12 1/12 1/6 1/6 1/6 1/6;
    1/24 1/24 1/12 1/12 1/12 1/6 1/6 1/6 1/6
]
# O_sub = zeros(num_ΔNTCP_states, num_observations)

13×9 Matrix{Float64}:
 0.166667   0.166667   0.166667   …  0.0833333  0.0416667  0.0416667
 0.166667   0.166667   0.166667      0.0833333  0.0416667  0.0416667
 0.166667   0.166667   0.166667      0.0833333  0.0416667  0.0416667
 0.0769231  0.0769231  0.153846      0.0769231  0.0769231  0.0769231
 0.0769231  0.0769231  0.153846      0.0769231  0.0769231  0.0769231
 0.0769231  0.0769231  0.0769231  …  0.153846   0.0769231  0.0769231
 0.0769231  0.0769231  0.0769231     0.153846   0.0769231  0.0769231
 0.0769231  0.0769231  0.0769231     0.153846   0.0769231  0.0769231
 0.04       0.08       0.08          0.16       0.16       0.08
 0.04       0.08       0.08          0.16       0.16       0.08
 0.0416667  0.0416667  0.0833333  …  0.166667   0.166667   0.166667
 0.0416667  0.0416667  0.0833333     0.166667   0.166667   0.166667
 0.0416667  0.0416667  0.0833333     0.166667   0.166667   0.166667

Check probabilities are valid

In [12]:
bad_rows = sanity_check_prob(O_sub) # currently has some rounding issues, hopefully will not break the solver

Tuple{Int64, Float64}[]

#### Create master observation probability matrix

In [26]:
#### Observation probabilities
# An observation is observed after the state transitions, hence depends on the new state
# O: <action> : <end-state> : <observation> %f

O = zeros(length(actions), num_states, num_observations);

Place probability submatrices into the master observation matrix

In [27]:
## Recall it is the observation observed at the end state

# First, have all end-states observe the dummy observation
for a in eachindex(actions)
    O[a, :, end] .= 1.0
end

# Next, update probabilities for end-states that shouldn't observe the dummy observation
## Replan
for t = 1:horizon
    for b = 1:min(t,horizon-1) # min(t,num_budget_states-1):num_budget_states-1
        ind_start = (t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states +1
        ind_end = (t)*num_budget_states*num_ΔNTCP_states +(b+1)*num_ΔNTCP_states
        O[1,ind_start:ind_end,1:end-1] = O_sub
        O[1,ind_start:ind_end,end] .= 0.0  # Remove observing the dummy state

        # foreach(i -> println("$(states[i])"), ind_start:ind_end)
    end
end
# println("\n\n\n")
## Continue
for t = 1:horizon
    for b = 1:min(t,horizon) # 1:num_budget_states
        ind_start = (t)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1
        ind_end = (t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states
        O[2,ind_start:ind_end,1:end-1] = O_sub
        O[2,ind_start:ind_end,end] .= 0.0  # Remove observing the dummy state

        # foreach(i -> println("$(states[i])"), ind_start:ind_end)
    end
end

Check probabilities are valid

In [37]:
bad_rows_R = sanity_check_prob(O[1,:,:])
# foreach(tup -> println("$(states[tup[1]])"), bad_rows_R)
foreach(tup -> println("$(states[tup[1]]) --- $(tup[2])"), bad_rows_R)

bad_rows_C = sanity_check_prob(O[2,:,:])
# foreach(tup -> println("$(states[tup[1]])"), bad_rows_C)
foreach(tup -> println("$(states[tup[1]]) --- $(tup[2])"), bad_rows_C)

Write observation matrix to an excel file (with labels)

In [ ]:
# Write observation probability matrix to an excel file (with labels)
# TODO

### Rewards

Immediate rewards

In [38]:
# R: <action> : <start-state> : <end-state> : <observation> %f

# Store master reward matrices in a dictionary to access by Action
R = Dict("Replan" => zeros(num_states, num_states, num_observations), "Continue" => zeros(num_states, num_states, num_observations))
# Rewards across ΔNTCP states
# TODO: Have we addresed the rewards from transitioning to a forbidden state?
# TODO: For example, at (b=0,t=4) and (b=0,t=5) cannot Replan. Is this acccounted for?
R_NTCP = [-(s_end - s_start) for s_start ∈ ΔNTCP_states, s_end ∈ ΔNTCP_states]

13×13 Matrix{Int64}:
  0  -1  -2  -3  -4  -5  -6  -7  -8  -9  -10  -11  -12
  1   0  -1  -2  -3  -4  -5  -6  -7  -8   -9  -10  -11
  2   1   0  -1  -2  -3  -4  -5  -6  -7   -8   -9  -10
  3   2   1   0  -1  -2  -3  -4  -5  -6   -7   -8   -9
  4   3   2   1   0  -1  -2  -3  -4  -5   -6   -7   -8
  5   4   3   2   1   0  -1  -2  -3  -4   -5   -6   -7
  6   5   4   3   2   1   0  -1  -2  -3   -4   -5   -6
  7   6   5   4   3   2   1   0  -1  -2   -3   -4   -5
  8   7   6   5   4   3   2   1   0  -1   -2   -3   -4
  9   8   7   6   5   4   3   2   1   0   -1   -2   -3
 10   9   8   7   6   5   4   3   2   1    0   -1   -2
 11  10   9   8   7   6   5   4   3   2    1    0   -1
 12  11  10   9   8   7   6   5   4   3    2    1    0

Scale R_NTCP by severity of each observation (if we'd like; otherwise, scale by 1 so that we only care about the ΔNTCP state)

In [39]:
# Recall we are assuming observations are ordered from best to worst, with high pain being worse than a high BMI drop
# The observation_intensities are to scale the rewards by the severity of the observation
observation_intensities = [i for i = num_observations:-1:1] # we are maximizing so decreasing values
# If we prefer rewards be independent of the observation (hence, no scaling necessary)
observation_intensities = [1 for _ = 1:num_observations]

## Replan
for t = 1:horizon
    for b = 1:num_budget_states-1
        row_indices = ((t-1)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t-1)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
        col_indices = ((t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states +1):((t)*num_budget_states*num_ΔNTCP_states +(b+1)*num_ΔNTCP_states)
        # println("\trow indices: $row_indices")
        # println("\tcol indices: $col_indices")
        for obs_ind in eachindex(observation_intensities)
            R["Replan"][row_indices, col_indices, obs_ind] = R_NTCP * observation_intensities[obs_ind]
        end
    end
end
## Continue
for t = 1:horizon
    for b = 1:num_budget_states
        row_indices = ((t-1)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t-1)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
        col_indices = ((t)*num_budget_states*num_ΔNTCP_states +(b-1)*num_ΔNTCP_states +1):((t)*num_budget_states*num_ΔNTCP_states +(b)*num_ΔNTCP_states)
        # println("\trow indices: $row_indices")
        # println("\tcol indices: $col_indices")
        for obs_ind in eachindex(observation_intensities)
            R["Continue"][row_indices, col_indices, obs_ind] = R_NTCP * observation_intensities[obs_ind]
        end
    end
end

Add remaining rewards

In [40]:
## Whenever the budget is 0 and action is Replan, give a big negative to reward to prevent this
for t = 1:horizon
    ind_start = (t-1)*num_ΔNTCP_states*num_budget_states + (num_budget_states-1)*num_ΔNTCP_states + 1
    ind_end = (t-1)*num_ΔNTCP_states*num_budget_states + (num_budget_states)*num_ΔNTCP_states
    # println("Range: $ind_start:$ind_end")
    # println("States: $(states[ind_start:ind_end])")

    # Reward is the same for any state and observation transitioning to the absorbing state (end)
    R["Replan"][ind_start:ind_end, end,:] .= -1000.0
end

## Rewards at the absorbing state (believe value shouldn't matter)
R["Replan"][end,end,:] .= 0.0
R["Continue"][end,end,:] .= 0.0

## Rewards at horizon + 1 (believe value shoulnd't matter)
range = horizon *num_ΔNTCP_states *num_budget_states +1:(horizon+1) *num_ΔNTCP_states *num_budget_states
R["Replan"][range, range,:] .= zeros(num_ΔNTCP_states*num_budget_states, num_ΔNTCP_states*num_budget_states);
R["Continue"][range, range,:] .= zeros(num_ΔNTCP_states*num_budget_states, num_ΔNTCP_states*num_budget_states);

## Write input file

#### Filename

In [43]:
# filename = "tester.POMDP"
filename = "tester-obs_indep_rewards_dummy_obs.POMDP"
# full_path = joinpath(pwd(), "Input Files", "$filename")
full_path = joinpath(pwd(), "$filename")

"/Users/raulgarcia/Documents/Rice University/Research/NIH SCH Adaptive Radiation Therapy/Aim 2_POMDP ART/Input Files/tester-obs_indep_rewards_dummy_obs.POMDP"

#### Write or append

In [44]:
f = open(full_path, "w")
# f = open(full_path, "a")

IOStream(<file /Users/raulgarcia/Documents/Rice University/Research/NIH SCH Adaptive Radiation Therapy/Aim 2_POMDP ART/Input Files/tester-obs_indep_rewards_dummy_obs.POMDP>)

#### Describe instance

In [45]:
write(f, "# Describe this instance here\n")
write(f, "# With dummy observation for unreachable states\n\n")

26

The first five line must be these (can be in a different order though)

In [46]:
#### Discount factor
write(f, "discount: $discount\n")

#### Rewards
write(f, "values: $values\n")

#### States
write(f, "states: $num_states\n")

#### Actions
write(f, "actions: ")
for act in actions
    write(f, "$act ")
end
write(f, "\n")

#### Observations
write(f, "observations: ")
for obs in observations
    write(f, "$obs ")
end
write(f, "\n\n")

flush(f)

After the initial five lines and optional starting state, the specifications of transition probabilities, 
observation probabilities and rewards appear. These specifications may appear in any order and can be intermixed.
Any probabilities or rewards not specified in the file are assumed to be zero. 
You may also specify a particular probability or reward more than once. The definition that appears last in the file 
is the one that will take affect. This is convenient for specifying exceptions to a more general specification.

#### Write starting state distribution (optional)

In [47]:
write(f, "start: ")
foreach(s_prob -> write(f, "$s_prob "), start_dist)
write(f, "\n\n")

2

#### Write state transition probabilitities

In [48]:
for a in eachindex(actions)
    write(f, "T: $(actions[a])\n")
    for s_start in Base.OneTo(num_states)
        for s_end in Base.OneTo(num_states)
            write(f, "$(T[a,s_start,s_end]) ")
        end
        write(f, "\n")
    end
    write(f, "\n")
end

#### Write observation matrix probabilities

In [49]:
for a in eachindex(actions)
    write(f, "O: $(actions[a])\n")
    for s_end in Base.OneTo(num_states)
        for o in Base.OneTo(num_observations)
            write(f, "$(O[a,s_end,o]) ")
        end
        write(f, "\n")
    end
    write(f, "\n")
end

#### Write immediate rewards

In [50]:
for action in actions, s_start in Base.OneTo(num_states)
    write(f, "R: $action : $(s_start-1)\n")
    for s_end in Base.OneTo(num_states)
        for o in Base.OneTo(num_observations)
            write(f, "$(R[action][s_start,s_end,o]) ")
        end
        write(f, "\n")
    end
    write(f, "\n")
end

### Close file

In [51]:
close(f)

## Execute pomdp-solve via command line

In [ ]:
# filename = "tester.POMDP"
filename = "tester-obs_indep_rewards_dummy_obs.POMDP"
cmd = `/Users/raulgarcia/Documents/pomdp-solve-5.5/src/pomdp-solve -pomdp $filename`

In [ ]:
run(cmd)

With optional paramters

In [ ]:
time_limit = 60  # seconds
cmd = `/Users/raulgarcia/Documents/pomdp-solve-5.5/src/pomdp-solve -pomdp $filename -time_limit $time_limit`
run(cmd)

In [ ]:
max_iter = 5
cmd = `/Users/raulgarcia/Documents/pomdp-solve-5.5/src/pomdp-solve -pomdp $filename -horizon $max_iter`
run(cmd)